In [2]:
import docplex
from docplex.mp.model import Model
import numpy as np
from sklearn.linear_model import LinearRegression

In [30]:
n = 5
M = 3
bundles = np.array([[0,1,1], [1,0,1]])
values = [3, 10]
solution = []
#np.random.randint(0, 2, size=(18, 10)).tolist()
reg = LinearRegression().fit(bundles, values)
bundle = [1,1,1]
coef = reg.coef_
intercept = reg.intercept_
print(np.sum(bundle * coef) + intercept)
#reg.predict([[1,1,1]])


#values = np.random.randint(0, 100, size=(18)).tolist()
#bundles = np.random.randint(0, 2, size=(18, 10)).tolist()
#reg = LinearRegression().fit(bundles, values)
#reg.predict([[0,1,1,0,0,0,0,0,0,0]])[0]

6.5


In [3]:
def galo_bids_init(value_model, bidder_id, n, presampled_n, presampled_algorithm, ml_model):

    values = np.random.randint(0, 100, size=(10)).tolist()
    bundles = np.random.randint(0, 2, size=(10, 5)).tolist()
    #print(bundles)
    M = 5

    # bundles random 18 bundles of length 10
    reg = LinearRegression().fit(bundles, values)

    r_max = []
    count = 10
    while count <= 20:
        coef = reg.coef_
        intercept = reg.intercept_
        for bundle in range(len(bundles)):

            m = Model(name='galo')
            r = m.continuous_var(name='r')
            
            x = m.binary_var_list(range(M), name='x')
            
            b = m.binary_var_list(len(values), name='b')


            constraints = []

            y_prediction = np.sum(x*coef) + intercept

            constraints.append(values[bundle] - y_prediction <= r)
            constraints.append(y_prediction - values[bundle] <= r)

            for j in range(len(values)):
                constraints.append(values[j] - y_prediction + 10000 * b[j] >= r)
                constraints.append(y_prediction - values[j] + 10000 * (1 - b[j]) >= r)

            m.add_constraints(constraints)
            m.maximize(r)

            sol = m.solve()

            try:
                sol.get_objective_value()
                print(sol)
            except:
                print('No solution found')
                #iterate over constraints
                for i in m.iter_constraints():
                    print(i)
                print('#################')
                #print(m.)
                #continue
        count += 1

    # D = np.array(bundles)
    # D = np.hstack((D, np.array(values).reshape(-1, 1)))
    # print(D)
    # return (D)
    return r_max

results = galo_bids_init(None, 0, 20, 10, 'unif', None)
#results

solution for: galo
r: 2.83851
r=2.839
x_0=1
x_4=1
b_1=1
b_2=1
b_9=1

solution for: galo
r: 10.4348
r=10.435
x_0=1
x_2=1
x_3=1
x_4=1
b_2=1

solution for: galo
r: 14.7516
r=14.752
x_3=1
x_4=1

solution for: galo
r: 4.0559
r=4.056
x_0=1
x_3=1
b_0=1
b_1=1
b_2=1
b_7=1
b_9=1

solution for: galo
r: 5.8323
r=5.832
x_0=1
x_1=1
x_2=1
b_0=1
b_1=1
b_2=1
b_3=1
b_4=1
b_5=1
b_6=1
b_7=1
b_8=1
b_9=1

solution for: galo
r: 6.27329
r=6.273
x_0=1
x_1=1
x_4=1
b_0=1
b_1=1
b_2=1
b_3=1
b_5=1
b_7=1
b_9=1

solution for: galo
r: 8.48447
r=8.484
x_1=1
b_0=1
b_1=1
b_2=1
b_3=1
b_5=1
b_7=1
b_8=1
b_9=1

solution for: galo
r: 4.75155
r=4.752
x_0=1
x_2=1
x_4=1
b_0=1
b_1=1
b_2=1
b_7=1
b_9=1

solution for: galo
r: 8.7205
r=8.720
x_0=1
x_2=1
b_0=1
b_1=1
b_2=1
b_3=1
b_5=1
b_7=1
b_8=1
b_9=1

solution for: galo
r: 10.4348
r=10.435
x_0=1
x_2=1
x_3=1
x_4=1
b_2=1

solution for: galo
r: 2.83851
r=2.839
x_0=1
x_4=1
b_1=1
b_2=1
b_9=1

solution for: galo
r: 10.4348
r=10.435
x_0=1
x_2=1
x_3=1
x_4=1
b_2=1

solution for: galo
r: 14.75

In [3]:
results

NameError: name 'results' is not defined